In [1]:
from new_model import * 

In [2]:
seed = 10
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## scheduler 학습
def fine_tuning(model, mode, EPOCHS):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed10_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
#     scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [7]:
# 모델 지정 및 GPU 지정
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 10, 0.3, 100).to(device)
check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x10_cifar100.pth")
model.load_state_dict(check_prune['state_dict'])
# test_model(model)

| Wide-Resnet 28x10


<All keys matched successfully>

In [8]:
INITIAL_LR = 0.1*0.2

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.4**0.1976

mode = '28x10_0.6_cosine_lr0.1*0.2'

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.6_CIFAR10_scheduler_multstep_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:3'), tensor(3736., device='cuda:3'))
Epoch 10: progress time is 2621.36 sec
Validation loss: 0.2937, Validation accuracy: 0.9212
Epoch 20: progress time is 5257.84 sec
Validation loss: 0.3055, Validation accuracy: 0.9088
Epoch 30: progress time is 7910.4 sec
Validation loss: 0.2639, Validation accuracy: 0.9220
Epoch 40: progress time is 10554.36 sec
Validation loss: 0.3218, Validation accuracy: 0.9136
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:3'), tensor(3112., device='cuda:3'))
Epoch 50: progress time is 13157.5 sec
Validation loss: 0.2958, Validation accuracy: 0.9140
Epoch 60: progress time is 15764.31 sec
Validation loss: 0.2865, Validation accuracy: 0.9156
Epoch 70: progress time is 18373.05 sec
Validation loss: 0.1471, Validation accuracy: 0.9588
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.6_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:2'), tensor(3736., device='cuda:2'))
Epoch 10: progress time is 2718.11 sec
Validation loss: 0.1445, Validation accuracy: 0.9580
Epoch 20: progress time is 5437.84 sec
Validation loss: 0.1586, Validation accuracy: 0.9552
Epoch 30: progress time is 8587.71 sec
Validation loss: 0.1478, Validation accuracy: 0.9580
Epoch 40: progress time is 11306.98 sec
Validation loss: 0.1442, Validation accuracy: 0.9592
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:2'), tensor(3112., device='cuda:2'))
Epoch 50: progress time is 14026.9 sec
Validation loss: 0.1555, Validation accuracy: 0.9560
Epoch 60: progress time is 16746.31 sec
Validation loss: 0.1414, Validation accuracy: 0.9620
Epoch 70: progress time is 19465.23 sec
Validation loss: 0.1426, Validation accuracy: 0.9616
Epoch 80: progr

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [10]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:2'), tensor(3736., device='cuda:2'))
Epoch 10: progress time is 2716.33 sec
Validation loss: 0.9059, Validation accuracy: 0.7804
Epoch 20: progress time is 5436.06 sec
Validation loss: 0.9158, Validation accuracy: 0.7736
Epoch 30: progress time is 8155.73 sec
Validation loss: 0.9061, Validation accuracy: 0.7748
Epoch 40: progress time is 10876.78 sec
Validation loss: 0.8794, Validation accuracy: 0.7792
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:2'), tensor(3112., device='cuda:2'))
Epoch 50: progress time is 13601.78 sec
Validation loss: 1.0685, Validation accuracy: 0.7508
Epoch 60: progress time is 16325.84 sec
Validation loss: 1.0243, Validation accuracy: 0.7624
Epoch 70: progress time is 19049.26 sec
Validation loss: 0.9109, Validation accuracy: 0.7828
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [10]:
##scheduler 실행 : cosineannealingLR, LR 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:0'), tensor(3736., device='cuda:0'))
Epoch 10: progress time is 2704.89 sec
Validation loss: 0.6634, Validation accuracy: 0.8124
Epoch 20: progress time is 5398.42 sec
Validation loss: 0.6556, Validation accuracy: 0.8220
Epoch 30: progress time is 8096.52 sec
Validation loss: 0.6825, Validation accuracy: 0.8100
Epoch 40: progress time is 10791.31 sec
Validation loss: 0.5824, Validation accuracy: 0.8368
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:0'), tensor(3112., device='cuda:0'))
Epoch 50: progress time is 13482.4 sec
Validation loss: 0.6756, Validation accuracy: 0.8228
Epoch 60: progress time is 16179.35 sec
Validation loss: 0.5817, Validation accuracy: 0.8408
Epoch 70: progress time is 18875.92 sec
Validation loss: 0.5916, Validation accuracy: 0.8456
Epoch 80: progres

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## seed 10 

In [22]:
##scheduler 실행 : cosineannealingLR, LR 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2534.37 sec
Validation loss: 0.0084, Validation accuracy: 0.9972
Epoch 20: progress time is 5066.22 sec
Validation loss: 0.0086, Validation accuracy: 0.9972
Epoch 30: progress time is 7596.62 sec
Validation loss: 0.0069, Validation accuracy: 0.9984
Epoch 40: progress time is 10127.01 sec
Validation loss: 0.0093, Validation accuracy: 0.9976
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 12636.4 sec
Validation loss: 0.0160, Validation accuracy: 0.9960
Epoch 60: progress time is 15145.37 sec
Validation loss: 0.0210, Validation accuracy: 0.9932
Epoch 70: progress time is 17649.66 sec
Validation loss: 0.0186, Validation accuracy: 0.9948
Epoch 80: progres

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosineannealingLR, LR 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:3'), tensor(3736., device='cuda:3'))
Epoch 10: progress time is 2703.66 sec
Validation loss: 0.1811, Validation accuracy: 0.9412
Epoch 20: progress time is 5398.49 sec
Validation loss: 0.1816, Validation accuracy: 0.9432
Epoch 30: progress time is 8092.91 sec
Validation loss: 0.1699, Validation accuracy: 0.9452
Epoch 40: progress time is 10790.46 sec
Validation loss: 0.1293, Validation accuracy: 0.9612
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:3'), tensor(3112., device='cuda:3'))
Epoch 50: progress time is 13549.34 sec
Validation loss: 0.1878, Validation accuracy: 0.9444
Epoch 60: progress time is 16239.66 sec
Validation loss: 0.1212, Validation accuracy: 0.9640
Epoch 70: progress time is 18929.25 sec
Validation loss: 0.1007, Validation accuracy: 0.9712
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##cifar100
##scheduler 실행 : cosineannealingLR, LR 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:1'), tensor(3736., device='cuda:1'))
Epoch 10: progress time is 2921.08 sec
Validation loss: 0.0889, Validation accuracy: 0.9832
Epoch 20: progress time is 6226.77 sec
Validation loss: 0.0795, Validation accuracy: 0.9848
Epoch 30: progress time is 9680.91 sec
Validation loss: 0.0770, Validation accuracy: 0.9864
Epoch 40: progress time is 12368.81 sec
Validation loss: 0.0821, Validation accuracy: 0.9840
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:1'), tensor(3112., device='cuda:1'))
Epoch 50: progress time is 15056.23 sec
Validation loss: 0.1882, Validation accuracy: 0.9592
Epoch 60: progress time is 17743.07 sec
Validation loss: 0.1610, Validation accuracy: 0.9672
Epoch 70: progress time is 20430.12 sec
Validation loss: 0.1674, Validation accuracy: 0.9668
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##cifar100
##scheduler 실행 : cosineannealingLR, LR 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:3'), tensor(4480., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3736., device='cuda:3'), tensor(3736., device='cuda:3'))
Epoch 10: progress time is 2698.95 sec
Validation loss: 0.7836, Validation accuracy: 0.7844
Epoch 20: progress time is 5390.27 sec
Validation loss: 0.8507, Validation accuracy: 0.7692
Epoch 30: progress time is 8080.4 sec
Validation loss: 0.7271, Validation accuracy: 0.8012
Epoch 40: progress time is 10769.69 sec
Validation loss: 0.6553, Validation accuracy: 0.8192
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(3112., device='cuda:3'), tensor(3112., device='cuda:3'))
Epoch 50: progress time is 13457.37 sec
Validation loss: 0.7641, Validation accuracy: 0.8032
Epoch 60: progress time is 16145.45 sec
Validation loss: 0.6075, Validation accuracy: 0.8396
Epoch 70: progress time is 18833.49 sec
Validation loss: 0.5755, Validation accuracy: 0.8472
Epoch 80: progres

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## 결과 확인

In [12]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.6_CIFAR10_scheduler_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9563; Test accuracy: 0.9563


In [11]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.6_CIFAR10_scheduler_multstep_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9546; Test accuracy: 0.9546


In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.6_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.6_CIFAR10_cosine finished')

Total examples is 10000, correct examples is 9582; Test accuracy: 0.9582


## CIFAR100

In [11]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.6_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.6_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7910; Test accuracy: 0.791


In [11]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.6_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7961; Test accuracy: 0.7961


## seed 10

### cifar10

In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9570; Test accuracy: 0.957


In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.6_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9569; Test accuracy: 0.9569


### cifar100

In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.6_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7887; Test accuracy: 0.7887


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.6_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7931; Test accuracy: 0.7931
